In [24]:
from sklearn.model_selection import train_test_split

#Validacion cruzada
from sklearn.model_selection import cross_val_score

#Escaladores (MinMax, Estandar)
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#Modelo de prediccion
from sklearn.neighbors import KNeighborsClassifier

#Metricas
from sklearn.metrics import confusion_matrix, classification_report

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("spaceship.csv")
df.tail()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False
8692,9280_02,Europa,False,E/608/S,TRAPPIST-1e,44.0,False,126.0,4688.0,0.0,0.0,12.0,Propsh Hontichre,True


In [3]:
df.shape

(8693, 14)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [5]:
df.isna().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [6]:
# Borramos las columnas que no necesitamos
df.drop(["PassengerId", "Name", "Cabin", "Destination"], axis=1, inplace=True)

In [7]:
df.shape

(8693, 10)

In [8]:
# Borramos valores nulos
df.dropna(inplace=True)
df.shape

(7074, 10)

## Preprocesamiento de datos

In [9]:
df.head()

,HomePlanet,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,16.0,False,303.0,70.0,151.0,565.0,2.0,True


### Variable dummy

In [10]:
# HomePlanet
homePlanet_dum = pd.get_dummies(df["HomePlanet"], prefix="HomePlanet")
df.drop("HomePlanet", axis=1, inplace=True)
df = df.join(homePlanet_dum)

# CrysoSleep
crysoSleep_dum = pd.get_dummies(df["CryoSleep"], prefix="CrysoSleep")
df.drop("CryoSleep", axis=1, inplace=True)
df = df.join(crysoSleep_dum)

# VIP
Vip_dum = pd.get_dummies(df["VIP"], prefix="VIP")
df.drop("VIP", axis=1, inplace=True)
df = df.join(Vip_dum)

df.head()

C:\Users\ninel\AppData\Local\Temp\ipykernel_13644\248013327.py:4: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  df = df.join(homePlanet_dum)


,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,CrysoSleep_False,CrysoSleep_True,VIP_False,VIP_True
0,39.0,0.0,0.0,0.0,0.0,0.0,False,0,1,0,1,0,1,0
1,24.0,109.0,9.0,25.0,549.0,44.0,True,1,0,0,1,0,1,0
2,58.0,43.0,3576.0,0.0,6715.0,49.0,False,0,1,0,1,0,0,1
3,33.0,0.0,1283.0,371.0,3329.0,193.0,False,0,1,0,1,0,1,0
4,16.0,303.0,70.0,151.0,565.0,2.0,True,1,0,0,1,0,1,0


In [11]:
df.shape

(7074, 14)

### Seleccion de datos

In [12]:
x = df.drop("Transported", axis=1)
y = df["Transported"]

In [13]:
# Standard Scaler
sc = StandardScaler()
x = sc.fit_transform(x)

In [14]:
x

array([[ 0.69595477, -0.33630897, -0.28775897, ..., -0.73656006,
         0.15739072, -0.15739072],
       [-0.34012695, -0.17403408, -0.28234362, ..., -0.73656006,
         0.15739072, -0.15739072],
       [ 2.00832495, -0.27229227,  1.86393791, ..., -0.73656006,
        -6.3536148 ,  6.3536148 ],
       ...,
       [-0.20198272, -0.33630897, -0.28775897, ..., -0.73656006,
         0.15739072, -0.15739072],
       [ 0.21244997, -0.33630897,  0.34342952, ..., -0.73656006,
         0.15739072, -0.15739072],
       [ 1.04131534, -0.14872515,  2.5330338 , ..., -0.73656006,
         0.15739072, -0.15739072]])

### Creacion del modelo

In [23]:
# Validacion cruzada
k = 3
knn_model = KNeighborsClassifier(n_neighbors=k)

scores = cross_val_score(knn_model, x, y, cv=5)
avg_score = np.mean(scores)

print(scores)
print(avg_score)

[0.74134276 0.75689046 0.75759717 0.75335689 0.76096181]
0.7540298179237408


In [25]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

In [26]:
knn_model = KNeighborsClassifier(n_neighbors=k)
knn_model.fit(x_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [27]:
y_pred = knn_model.predict(x_test)

### Evaluacion del modelo

In [28]:
cf_matrix = confusion_matrix(y_test, y_pred)
cf_matrix

array([[535, 150],
       [189, 541]], dtype=int64)

In [29]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.74      0.78      0.76       685
        True       0.78      0.74      0.76       730

    accuracy                           0.76      1415
   macro avg       0.76      0.76      0.76      1415
weighted avg       0.76      0.76      0.76      1415



## Guardar modelo en PC

In [31]:
import pickle

In [32]:
nombre = "Modelo_clasificacion_spaceship.sav"
pickle.dump(knn_model, open(nombre, "wb"))